## Importing Cooling calculator

In [ ]:
# CHANGE UNIT_DENSITY IN THE CODE BEFORE IMPORTING IT

%cd /home/artur/Desktop/Radiative Cooling/Tables_cooling
import cooling_table as cool

## Opening work directory

In [ ]:
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1.5e-6/
%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-7/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho3e-7/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-9/

## Importing necessary packages

In [ ]:
import mickey.mickey
import mickey.plot
import tqdm
import numpy as np

## Useful constants

In [ ]:
#Mass of black hole in solar mass
mbh = 10.0

#units of sim
unit_length = 2.969707e6
unit_velocity = 2.1198528e10

#MOST IMPORTANT UNIT TO MY WORK
unit_density = 1e-6

#Eddington luminosity for mbh in cgs
Ledd = 1.26e38*mbh

#Constants
#mu = 0.64494
kelvin = 5.405e12
beta = 10.0
amu = 1.66053886e-24
mu_e = 1.142857143
mu_i = 1.230769231

In [ ]:
from scipy.special import kn
from scipy.optimize import root
import numpy as np

# Constants to be used in cgs
C_sigma = 5.67051e-5#Stephan Boltzmann constant
C_kb = 1.3806505e-16#Boltzmann constant
C_h = 6.62606876e-27#Planck constant
C_me = 9.1093826e-28#Electron mass
C_mp = 1.67262171e-24#Proton mass
C_amu = 1.66053886e-24#Atomic mass unit
C_c = 2.99792458e10#Speed of light
C_G = 6.6726e-8#Gravitational constant
C_Msun = 2.0e33#Sun mass
C_sigmaT = 6.6524e-25#Thomson cross section
#C_pi = 3.14159265358979 #PI

#Mass of the Black hole
C_Mbh = 10. * C_Msun
C_GM = C_Mbh * C_G

#Schwarzschild Radius
Rs = 2. * C_G * C_Mbh / (C_c * C_c)

#Adiabatic index
C_gamma = 5./3.

#Constant of temperature parametrization
CONST_1 = C_kb / (C_me * C_c * C_c)

#Temperature parametrization
def theta_e (Te):
    return (CONST_1 * Te)

#Disk geometry and polytropic constant
rho_0 = 5e-7#Maximum density of initial condition
r_0 = 100. * Rs #Radius of maximum density (rho_0)
r_min = 75. * Rs #Minimum raius of torus
CONST_2 = - C_GM/(r_min-Rs) + C_GM/(2.*r_min*r_min)*(r_0*r_0*r_0)/((r_0-Rs)*(r_0-Rs))
kappa = (C_gamma-1.)/C_gamma*pow(rho_0, 1.-C_gamma)*(CONST_2 + C_GM/(r_0-Rs) 
        - C_GM/2. * r_0/((r_0-Rs)*(r_0-Rs))) #Polytropic constant

#Entangled magnetic field (local randomly oriented magnetic field)
beta = 10.

#Sound speed cs = sqrt(d P/d rho)
def sound_speed(ne):
    result = C_gamma * kappa * pow(ne * 1.14 * C_amu, C_gamma-1.)
    result = np.sqrt(result)
    return result

#Magnetic field assuming equipartition
def B(ne):
    result = 8.*np.pi*sound_speed(ne)*sound_speed(ne)*ne*1.14*C_amu/(beta+1.)
    result = np.sqrt(result)
    return result

#Scale Height following the expression cs/omega_K
def scale_height(R, ne):
    #result = np.sqrt(R/C_GM)*sound_speed(ne)*(R-Rs)
    return R

#BREEMSTRAHLUNG PART
#Electron-ion collision
def Fei(Te):
    th_e = theta_e(Te)

    return np.where(np.asarray(th_e) >= 1.0, 9.*th_e/(2.*np.pi)*(np.log(1.123*th_e+0.48)+1.5), 4.*np.sqrt(2.*th_e/(np.pi**3.))*(1.+1.781*th_e**(1.34)))

#Electron-ion colling rate
def Qei(ne, Te):
    result = 1.48e-22
    result *= (ne*ne*Fei(Te))

    return result

#Electron-Electron collision
def Qee(ne, Te):

    th_e = theta_e(Te)

    return np.where(np.asarray(th_e) <= 1.0, 2.56e-22*(ne*ne*pow(th_e, 1.5))*(1.+1.1*th_e+th_e*th_e-1.25*pow(th_e, 2.5)),
            3.4e-22*(ne*ne*th_e)*(np.log(1.123*th_e)+1.28))

#Breemstralung cooling rate
def Qbrem(ne, Te):
    result = Qei(ne, Te) + Qee(ne, Te)
    return result


#SYNCHROTRON PART
#Function for simplicity, using bessel function
def BTB(Te, ne):
    result = kn(2,1./theta_e(Te)) * theta_e(Te) * theta_e(Te) * theta_e(Te) * B(ne)
    result = 1. / result
    return result

#Transcedental equation for xm
def TransEq(x, Te, R, ne):
    result = 1./x**(7./6.) + 0.4/x**(17./12.) + 0.5316/x**(5./3.)
    result *= 2.49e-10*12*np.pi*ne*scale_height(R, ne)*BTB(Te, ne)
    result -= np.exp(1.8899*x**(1./3.))
    return result

#critical frequency
def nu_c(ne, Te, R):
    xm = [[root(TransEq, 1.e-3, args=(Te[i][j], R[i][j], ne[i][j])).x[0] for j in range(length)] for i in range(length)]
    #print(xm.x)
    result = 3. * 2.8e6 * B(ne) * theta_e(Te) * theta_e(Te) * xm/2.
    return result

#Synchrotron cooling rate
def Qsyn(ne, R, Te):
    result = nu_c(ne, Te, R)
    result = result * result * result
    result = 2. * np.pi * C_kb * Te * result
    result = result/(3. * C_c * C_c * R)
    return result


#SYNCHROTRON SELF COMPTON PART
#Scattering optical depth
def tau_es(R, ne):
    result = 2. * ne * C_sigmaT * scale_height(R, ne)
    return result

#Mean amplification factor in the energy of the scattered photon when scattering electrons 
#have a Maxwellian velocity distribution of temperature
def Amp(Te):
    return (1. + 4.*theta_e(Te) + 16.*theta_e(Te)*theta_e(Te))

#Energy normalization
def enorm(ne, Te, R):
    result = C_h * nu_c(ne, Te, R)
    result = result/(C_me * C_c * C_c)
    return result

#Probability of scattering a photon
def Prob(R, ne):
    return 1. - np.exp(-tau_es(R, ne))

#Comptonized energy anhancement factor
def eta(ne, R, Te):
    eta1 = Prob(R, ne) * (Amp(Te)-1.)
    eta1 = eta1 / (1. - Prob(R, ne)*Amp(Te))
    eta3 = -1. - np.log(Prob(R, ne))/np.log(Amp(Te))
    result = 1. + eta1 - eta1*(enorm(ne, Te, R)/(3.*theta_e(Te)))**(eta3)
    return result

#Synchrotron self compton cooling rate
def Qssc(ne, R, Te):
    result = Qsyn(ne, R, Te)*(eta(ne, R, Te) - 1.)
    return result

#Total cooling rate in optically thin approximation
def Q1(ne, R, Te):
    result = Qbrem(ne, Te) + Qsyn(ne, R, Te) + Qssc(ne, R, Te)
    return result

#Absorption optical depth
def tau_abs(ne, R, Te):
    result = scale_height(R, ne) * Q1(ne, R, Te)
    result = result/(4.*C_sigma*Te*Te*Te*Te)
    return result

#total optical depth in the vertical direction from the disk midplane surface
def tau_tot(ne, R, Te):
    result = tau_abs(ne, R, Te) + tau_es(R, ne)
    return result

#Resulting cooling rate for both optically thick and optically thin cooling limits
def Qtot(ne, R, Te):
    result = 4. * C_sigma * Te * Te * Te * Te / scale_height(R, ne)
    result = result / (3.*tau_tot(ne, R, Te)/2. + np.sqrt(3.) + 1./tau_abs(ne, R, Te))
    return result

In [ ]:
p = mickey.mickey.Pluto(2400,stdout=False)

In [ ]:
ne = p.rho * unit_density/ (mu_e * amu)

In [ ]:
r = p.X1 * unit_length

In [ ]:
te = beta/(beta + 1.0) * p.p/p.rho * kelvin * mu_e
te /= (1.0 + mu_e/mu_i * ((p.X1/100.)**(-1) + 2.0))

In [ ]:
length = len(ne)

In [ ]:
xm = [[root(TransEq, 1.e-3, args=(te[i][j], r[i][j], ne[i][j])).x[0] for j in range(length)] for i in range(length)]
#print(nu_c(ne, te, r))
#print(np.shape(nu_c(ne, te, r)))

In [ ]:
xm = np.array(xm)

In [ ]:
print(xm)

In [ ]:
%time cooling_rate = np.where(p.tr1 >= 0.99, Qtot(ne, r, te), 0.0)

## Calculating cooling rate and luminosity

In [ ]:
def Lum(ne, r, te, dr, theta, dtheta, tr1, length, i):
    #luminosity = 0.0
    #cooling_rate = np.zeros((400,400), dtype=float)
    brem = np.zeros((400,400), dtype=float)
    syn = np.zeros((400,400), dtype=float)
    ssc = np.zeros((400,400), dtype=float)
    
    for j in range(length):
        if tr1[i][j] > 0.99:
            brem[i][j] = cool.Qbrem(ne[i][j], te[i][j])
            syn[i][j] = cool.Qsyn(ne[i][j], r[i], te[i][j])
            ssc[i][j] = cool.Qssc(ne[i][j], r[i], te[i][j])
            #luminosity += cooling_rate[i][j] * r[i] * r[i] * np.sin(theta[j]) * dr[i] * dtheta[j]
    return (brem[i], syn[i], ssc[i])

In [ ]:
import multiprocessing as mp
nprocs = mp.cpu_count()
pool = mp.Pool(processes=nprocs)

In [ ]:
#first snap, last snap, length of array
snapi = 2123
snapf = 2124
length = 400

#luminosity = np.zeros((snapf-snapi), dtype=float)

for k in tqdm.tqdm(range(snapi, snapf)):
    
    #cooling_rate = np.zeros((400,400), dtype=float)
    #brem = np.zeros((400,400), dtype=float)
    #syn = np.zeros((400,400), dtype=float)
    #ssc = np.zeros((400,400), dtype=float)
    
    p = mickey.mickey.Pluto(k,stdout=False)
    
    #Calculating variables
    #distance
    r = p.x1 * unit_length

    #electronic density
    ne = p.rho * unit_density/ (mu_e * amu)

    #eletronic temperature
    temperature = beta/(beta + 1.0) * p.p/p.rho * kelvin * mu_e
    te = []
    for j in range(len(temperature)):
        te.append(temperature[j]/(1.0 + mu_e/mu_i * ((p.x1[j]/100.)**(-1) + 2.0)))
    te = np.array(te)
    
    #delta r
    dr = p.dx1 * unit_length
    
    #index
    l = k-snapi
    
    #calculating an array to calculate luminosity
    cooling_rate = pool.starmap(Lum, [(ne, r, te, dr, p.x2, p.dx2, p.tr1, length, i) for i in range(length)])
    
    #calculating luminosity for each snapshot
    #luminosity[l] = 2.*np.pi*sum(result)/Ledd

In [ ]:
np.savez('luminosity.npz', lum=luminosity)

In [ ]:
data=np.load('luminosity.npz')
luminosity=data['lum']

In [ ]:
print(np.median(luminosity), np.std(luminosity))

In [ ]:
%pylab inline

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(luminosity, bins=20, density=True)
plt.axvline(np.median(luminosity), color='k', linewidth=2)
plt.axvline((np.median(luminosity) + np.std(luminosity)), color='r', linestyle='dashed', linewidth=2)
plt.axvline((np.median(luminosity) - np.std(luminosity)), color='r', linestyle='dashed', linewidth=2)

min_ylim, max_ylim = plt.ylim()
plt.text(np.median(luminosity)*1.3, max_ylim*0.9, 'Median: {:.2f}'.format(np.median(luminosity)), fontsize=18)

xlabel("$L/L_{edd}$", fontsize=15)
ylabel("Frequency", fontsize=15)

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

tight_layout()

plt.show()

In [ ]:
#cooling_rate = np.array(cooling_rate)
b = np.array([cooling_rate[i][0] for i in range(400)])
syn = np.array([cooling_rate[i][1] for i in range(400)])
ssc = np.array([cooling_rate[i][2] for i in range(400)])
syntot = syn+ssc
tot = b + syntot

In [ ]:
print(np.amin(b/tot))

In [ ]:
plt.figure(figsize=(10,10))

pcolormesh(p.X, p.Y, (b/tot).T, vmin=0, vmax=1., cmap='jet')
#pcolormesh(p.X, p.Y, log10(ne.T), vmin=log10(np.amin(ne)), vmax=log10(np.amax(ne)), cmap='jet')
#pcolormesh(p.X, p.Y, p.tr1.T, vmin=0.1, vmax=1., cmap='jet')
xlabel("$R/R_{s}$", fontsize=15)
ylabel("$Z/R_{s}$", fontsize=15)
plt.title("$Q^{-}$ (erg s$^{-1}$ cm$^{-3}$)", fontsize=16)
plt.xlim(0, 100)
plt.ylim(-100/2., 100/2.)
circle2=Circle((0,0),1.,color='k')
gca().add_artist(circle2)
colorbar()

tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

tight_layout()

plt.show()
plt.clf()
plt.cla()
plt.close('all')

## Sanity Check of luminosity